# Three Generations Seeking Romance

<img src="https://teknodestek.com.tr/wp-content/uploads/2023/05/Okcupid-Hesap-Nasil-Silinir.png" width="300px">

Online randevulara ait yaşa ve nesile göre profil oluşturulaması projesi. Bu projede aşağıdaki sorulara yanıt aranacaktır:

1. Flört profiliniz yaşınızı tahmin edebilir mi?
2. Flört profiliniz ait olduğunuz kuşağı tahmin edebiliyor mu? (Y Kuşağı-Millennial, X Kuşağı-Gen X-er veya Boomers)
<br>

Arkadaşlık platformu OKCupid'den yaklaşık 60.000 anonimleştirilmiş girişten oluşan bir veri kümesi kullanarak, yukarıdaki soruları denetimli makine öğrenimi tekniklerini kullanarak keşfedeceğiz. Bu projede iki model kullanacağız:
* `Regression`: İlk soruyu ele almak için regresyon teknikleri kullanacağız.
* `Classification`: İkinci soruyu ele almak için ise sınıflandırma modellerini kullanacağız.

- "last_online" özelliğinin minimum ve maksimum değerleri incelenerek OKCupid veri kümesinin 2011 veya 2012 civarına tarihlendiği sonucuna varılabilir. 
- O zamanlar kuşaklar aşağıdaki gibidir:
  - Y Kuşağı 18-32
  - X kuşağı 33-47 
  - Boomers  48-70uşağı 33-47 
  - Boomers  48-70e kadar.dar.

### Veri Setini İnceleyelim

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 100

import warnings
warnings.filterwarnings('ignore')

import numpy as np

In [2]:
df=pd.read_csv("profiles.csv")
df.head()

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22.0,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...,"asian, white",75.0,-1.0,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35.0,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN,white,70.0,80000.0,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38.0,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",NaN,68.0,-1.0,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
3,23.0,thin,vegetarian,socially,NaN,working on college/university,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . .<br />\nlynch, j...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,white,71.0,20000.0,student,2012-06-28-14-22,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single
4,29.0,athletic,NaN,socially,never,graduated from college/university,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at:<br />\nhttp://bag...,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians<br />\nat the...",NaN,NaN,NaN,NaN,NaN,"asian, black, other",66.0,-1.0,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60552 entries, 0 to 60551
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          9514 non-null   float64
 1   body_type    8666 non-null   object 
 2   diet         5761 non-null   object 
 3   drinks       9012 non-null   object 
 4   drugs        7215 non-null   object 
 5   education    8459 non-null   object 
 6   essay0       8667 non-null   object 
 7   essay1       8347 non-null   object 
 8   essay2       8049 non-null   object 
 9   essay3       7690 non-null   object 
 10  essay4       7879 non-null   object 
 11  essay5       7814 non-null   object 
 12  essay6       7352 non-null   object 
 13  essay7       7553 non-null   object 
 14  essay8       6361 non-null   object 
 15  essay9       7554 non-null   object 
 16  ethnicity    8565 non-null   object 
 17  height       9514 non-null   float64
 18  income       9514 non-null   float64
 19  job 

In [4]:
# Neredeyse hiç dolu alan yok. Bu veri seti çok uğraştıracağa benziyor :)
df.isnull().sum()

age            51038
body_type      51886
diet           54791
drinks         51540
drugs          53337
education      52093
essay0         51885
essay1         52205
essay2         52503
essay3         52862
essay4         52673
essay5         52738
essay6         53200
essay7         52999
essay8         54191
essay9         52998
ethnicity      51987
height         51038
income         51038
job            52369
last_online    51038
location       51038
offspring      56666
orientation    51038
pets           54200
religion       54291
sex            51038
sign           52776
smokes         51956
speaks         51044
status         51038
dtype: int64

In [5]:
# Önce tüm alanları boş olan satırları silelim
df = df.dropna(how='all')

In [6]:
# Bakalım hangi alanda kaç adet boş kayıt var, sonucu 0 olmayan alanlar
miss = df.isna().sum()
miss[miss > 0]

body_type     848
diet         3753
drinks        502
drugs        2299
education    1055
essay0        847
essay1       1167
essay2       1465
essay3       1824
essay4       1635
essay5       1700
essay6       2162
essay7       1961
essay8       3153
essay9       1960
ethnicity     949
job          1331
offspring    5628
pets         3162
religion     3253
sign         1738
smokes        918
speaks          6
dtype: int64

Şimdi oldu. Artık işimize bakabiliriz

### EDA - Keşfedici Veri Analizi
Bu aşamada her bir alan için aşağıdaki işlemler yapılacaktır:
* Veri yapısını incele
* Boşlukları doldur
* Özellik mühendisliği ile yeni alanlar oluştur
* Object türünü kategoriye çevir

In [7]:
# Veri tanıma sürecinde belirlenen bazı alanlara toplu işlem yapılması planlanmaktadır.
# Bu amaçla aşağıdaki dizilere eklenecek alanlar Veri ön işleme aşamasında kullanılacaktır
obj_cols=[] # Toplu bir şekilde bu listedeki alanlar kategoriye çevrilecektir
drp_cols=[] # Bu listedeki alanlar modele (x) dahil edilmeyeceklerdir

In [8]:
df["body_type"].value_counts()

body_type
average           2274
fit               2040
athletic          1888
thin               740
curvy              626
a little extra     430
skinny             269
full figured       181
overweight          69
jacked              64
used up             47
rather not say      38
Name: count, dtype: int64

In [9]:
# Boşlukları önce "rather not say" ile dolduralım. Sonra gruplayalım
df.body_type.fillna("rather not say", inplace=True)
body_type_mapping = {
    'average': 'athletic',
    'fit': 'athletic',
    'athletic': 'athletic',
    'thin': 'thin',
    'curvy': 'curvy',
    'a little extra': 'curvy',
    'skinny': 'thin',
    'full figured': 'curvy',
    'overweight': 'curvy',
    'jacked': 'athletic',
    'used up': 'curvy',
    'rather not say': 'other'
}
df['body_type'] = df['body_type'].map(body_type_mapping)
obj_cols.append("body_type")

In [10]:
df["diet"].value_counts()

diet
mostly anything        2621
anything               1067
strictly anything       835
mostly vegetarian       568
mostly other            154
strictly vegetarian     135
vegetarian              111
strictly other           64
mostly vegan             52
other                    52
strictly vegan           40
vegan                    28
mostly halal             13
mostly kosher            12
strictly halal            4
strictly kosher           2
kosher                    2
halal                     1
Name: count, dtype: int64

In [11]:
# Önce diet alanındaki "mostly" ve "strictly" ifadelerini silelim
df['diet'] = df['diet'].str.replace('mostly ', '').str.replace('strictly ', '')
# Boşlukları önce "anything" sonra da "other" ile doldurup başarı durumuna bakalım
df.diet.fillna("other", inplace=True)
obj_cols.append("diet")

In [12]:
df['drinks'].value_counts()

drinks
socially       6679
rarely          911
often           809
not at all      509
very often       54
desperately      50
Name: count, dtype: int64

In [13]:
# Sayı olarak işe yaramadı bir de kategori deneyelim
#df.drinks.fillna("not at all", inplace=True)
#df['drinks'] = df['drinks'].apply(lambda x: 0 if pd.isnull(x) or x == 'not at all' else 1)
df.drinks.fillna("socially", inplace=True)
obj_cols.append("drinks")

In [14]:
df.drugs.value_counts()

drugs
never        5972
sometimes    1181
often          62
Name: count, dtype: int64

In [15]:
df.drugs.fillna("never", inplace=True)
obj_cols.append("drugs")
#df["drugs"]=np.where(df["drugs"]=="never",0,1)

In [16]:
df.education.value_counts()

education
graduated from college/university    3770
graduated from masters program       1402
working on college/university         969
working on masters program            251
graduated from two-year college       245
graduated from high school            205
graduated from ph.d program           203
graduated from law school             177
working on two-year college           159
working on ph.d program               159
dropped out of college/university     149
college/university                    134
graduated from space camp             104
dropped out of space camp              87
working on space camp                  73
graduated from med school              73
working on law school                  55
two-year college                       43
working on med school                  41
dropped out of two-year college        34
masters program                        21
working on high school                 20
dropped out of ph.d program            19
dropped out of masters p

In [17]:
# Eğitim düzeyini kategorilere ayırma
df['education_category'] = df['education'].replace({
    'graduated from college/university': 'Higher Education',
    'graduated from masters program': 'Higher Education',
    'working on college/university': 'Higher Education',
    'working on masters program': 'Higher Education',
    'graduated from two-year college': 'Higher Education',
    'graduated from high school': 'Secondary Education',
    'graduated from ph.d program': 'Higher Education',
    'graduated from law school': 'Higher Education',
    'working on two-year college': 'Higher Education',
    'working on ph.d program': 'Higher Education',
    'dropped out of college/university': 'Higher Education',
    'college/university': 'Higher Education',
    'graduated from space camp': 'Other',
    'dropped out of space camp': 'Other',
    'working on space camp': 'Other',
    'graduated from med school': 'Other',
    'working on law school': 'Other',
    'two-year college': 'Higher Education',
    'working on med school': 'Other',
    'dropped out of two-year college': 'Higher Education',
    'masters program': 'Higher Education',
    'working on high school': 'Secondary Education',
    'dropped out of ph.d program': 'Higher Education',
    'dropped out of masters program': 'Higher Education',
    'dropped out of high school': 'Secondary Education',
    'high school': 'Secondary Education',
    'space camp': 'Other',
    'ph.d program': 'Higher Education',
    'law school': 'Other',
    'dropped out of law school': 'Other',
    'dropped out of med school': 'Other'
})

In [18]:
df['education_category'].value_counts()

education_category
Higher Education       7756
Other                   452
Secondary Education     251
Name: count, dtype: int64

In [19]:
df['education_category'].fillna("Other", inplace=True)

In [20]:
# Eğitim durumu için yeni bir alan oluşturalım ama hata verdiği için önce alanı stringe çevirelim
df['education'] = df['education'].astype(str)
df['education_status'] = df['education'].apply(lambda x: 'graduated' if 'graduated' in x
                                               else ('working on' if 'working on' in x
                                                     else 'dropped out of' if 'dropped out' in x
                                                     else 'unknown'))

In [21]:
df['education_status'].value_counts()

education_status
graduated         6179
working on        1727
unknown           1285
dropped out of     323
Name: count, dtype: int64

In [22]:
drp_cols.append("education")
obj_cols.append("education_category")
obj_cols.append("education_status")

In [23]:
 df['ethnicity'].value_counts()

ethnicity
white                                                                5278
asian                                                                 863
hispanic / latin                                                      426
black                                                                 326
other                                                                 319
                                                                     ... 
native american, pacific islander, hispanic / latin, white, other       1
indian, pacific islander                                                1
asian, middle eastern, black                                            1
asian, middle eastern, indian                                           1
middle eastern, hispanic / latin, white, other                          1
Name: count, Length: 112, dtype: int64

In [24]:
# Liste kalabalık biraz törpüleyelim
# Ana Etnik Grup bize yeter gibi
df['ethnicity'] = df['ethnicity'].astype(str)
df['primary_ethnic_group'] = df['ethnicity'].apply(lambda x: x.split(', ')[0])
df['primary_ethnic_group'].value_counts()

primary_ethnic_group
white               5384
asian               1143
nan                  949
hispanic / latin     668
black                496
other                319
indian               205
middle eastern       141
native american      111
pacific islander      98
Name: count, dtype: int64

In [25]:
# Alanı stringe çevirince boşluklar "nan" olmuş. Boşlukları önce white sonra other ile dolduralım
df['primary_ethnic_group']=df['primary_ethnic_group'].str.replace("nan", "white")

In [26]:
drp_cols.append("ethnicity")
obj_cols.append("primary_ethnic_group")

In [27]:
df.job.value_counts()

job
other                                1153
student                               807
science / tech / engineering          780
artistic / musical / writer           715
computer / hardware / software        709
sales / marketing / biz dev           709
medicine / health                     613
education / academia                  538
executive / management                378
entertainment / media                 356
banking / financial / real estate     353
hospitality / travel                  217
law / legal services                  200
construction / craftsmanship          178
clerical / administrative             117
political / government                105
rather not say                         68
transportation                         64
unemployed                             56
retired                                38
military                               29
Name: count, dtype: int64

In [28]:
# Durumu daha özet hale getiren yeni bir alan oluşturalım
df['job_status'] = np.where(df['job'] == 'student', 'Student', np.where(df['job'] == 'unemployed', 'Unemployed', 'Employed'))

In [29]:
df.job_status.value_counts()

job_status
Employed      8651
Student        807
Unemployed      56
Name: count, dtype: int64

In [30]:
drp_cols.append("job")
obj_cols.append("job_status")

In [31]:
df.offspring.value_counts()

offspring
doesn&rsquo;t have kids                                1224
doesn&rsquo;t have kids, but might want them            620
doesn&rsquo;t have kids, but wants them                 578
doesn&rsquo;t want kids                                 454
has a kid                                               304
has kids                                                291
doesn&rsquo;t have kids, and doesn&rsquo;t want any     172
has kids, but doesn&rsquo;t want more                    68
has a kid, but doesn&rsquo;t want more                   41
has a kid, and might want more                           36
wants kids                                               29
might want kids                                          28
has kids, and might want more                            24
has a kid, and wants more                                12
has kids, and wants more                                  5
Name: count, dtype: int64

In [32]:
# Önce boşlukları dolduralım
df['offspring'].fillna("doesn&rsquo;t have kids", inplace=True)

In [33]:
# Yine liste Uzun kısaltalım.
df['has_kid'] = 0
df['wants_kid'] = 1

# "has children" ve "doesn't have kids, but wants them" değerlerini 1 olarak güncelleme
df.loc[df['offspring'].str.contains('has'), 'has_kid'] = 1
df.loc[df['offspring'].str.contains('doesn&rsquo;t want'), 'wants_kid'] = 0

In [34]:
drp_cols.append("offspring")

In [35]:
df.pets.value_counts()

pets
likes dogs and likes cats          2314
likes dogs                         1152
has dogs                            683
likes dogs and has cats             674
has dogs and likes cats             384
likes dogs and dislikes cats        301
has dogs and has cats               226
has cats                            221
likes cats                          200
has dogs and dislikes cats           82
dislikes dogs and dislikes cats      42
dislikes dogs and likes cats         36
dislikes cats                        20
dislikes dogs and has cats           13
dislikes dogs                         4
Name: count, dtype: int64

In [36]:
# Hayvan sever ve sahip olma durumlarını ayıralım
df['likes_animals'] = 0
df['has_pets'] = 0
df['pets'] = df['pets'].astype(str)

# "likes" ifadesini içeren değerleri 1 olarak güncelleme
df.loc[df['pets'].str.contains('likes|has'), 'likes_animals'] = 1

# "has" ifadesini içeren değerleri 1 olarak güncelleme
df.loc[df['pets'].str.contains('has'), 'has_pets'] = 1

In [37]:
drp_cols.append("pets")

In [38]:
df.religion.value_counts()

religion
agnosticism                                   448
other                                         417
agnosticism but not too serious about it      400
agnosticism and laughing about it             386
catholicism but not too serious about it      367
atheism                                       363
atheism and laughing about it                 329
other and laughing about it                   325
christianity but not too serious about it     320
christianity                                  297
other but not too serious about it            243
judaism but not too serious about it          232
atheism but not too serious about it          200
catholicism                                   165
atheism and somewhat serious about it         138
christianity and somewhat serious about it    133
other and somewhat serious about it           122
catholicism and laughing about it             117
judaism and laughing about it                 113
agnosticism and somewhat serious about it

In [39]:
# Önce boşlukları other ile dolduralım
df.religion.fillna("other", inplace=True)
# Sadece din kısmını alalım
df['religion'] = df['religion'].apply(lambda x: x.split()[0])
df['religion'].value_counts()

religion
other           4446
agnosticism     1381
atheism         1131
christianity     905
catholicism      762
judaism          490
buddhism         288
hinduism          82
islam             29
Name: count, dtype: int64

In [40]:
obj_cols.append("religion")

In [41]:
df.smokes.value_counts()

smokes
no                7011
sometimes          565
when drinking      480
yes                307
trying to quit     233
Name: count, dtype: int64

In [42]:
df.smokes.fillna("no")

0           sometimes
1                  no
2                  no
3                  no
4                  no
            ...      
9509               no
9510        sometimes
9511    when drinking
9512               no
9513               no
Name: smokes, Length: 9514, dtype: object

In [43]:
# Sigara içenlere 1, içmeyenlere 0 verelim
df['smokes'] = (df['smokes'] != 'no').astype(int)
# Bu kısımda boşlukları da 1 ile doldurmuş olduk. Daha sonra buraya dönüp 0 ile doldurmak gerekebilir

In [44]:
miss = df.isna().sum()
miss[miss > 0]

essay0     847
essay1    1167
essay2    1465
essay3    1824
essay4    1635
essay5    1700
essay6    2162
essay7    1961
essay8    3153
essay9    1960
job       1331
sign      1738
speaks       6
dtype: int64

In [45]:
# Şimdilik yukarıdaki alanları da modele dahil etmeyelim. Sonra duruma göre bakarız.
drp_cols.extend(["essay0", "essay1", "essay2", "essay3", "essay4", "essay5", "essay6", "essay7", "essay8", "essay9", "sign", "speaks"])
drp_cols

['education',
 'ethnicity',
 'job',
 'offspring',
 'pets',
 'essay0',
 'essay1',
 'essay2',
 'essay3',
 'essay4',
 'essay5',
 'essay6',
 'essay7',
 'essay8',
 'essay9',
 'sign',
 'speaks']

In [46]:
df.orientation.value_counts()

orientation
straight    8191
gay          897
bisexual     426
Name: count, dtype: int64

In [47]:
obj_cols.append("orientation")

In [48]:
df.sex.value_counts()

sex
m    5675
f    3839
Name: count, dtype: int64

In [49]:
obj_cols.append("sex")

In [50]:
# Birkaç alanı daha ihmal edelim
drp_cols.extend(["last_online", "location"])
# Gereksiz alanları veri setinden çıkaralım
df.drop(drp_cols, axis=1, inplace=True)
# Tüm objeleri kategori yapalım
df[obj_cols] = df[obj_cols].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9514 entries, 0 to 9513
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   age                   9514 non-null   float64 
 1   body_type             9514 non-null   category
 2   diet                  9514 non-null   category
 3   drinks                9514 non-null   category
 4   drugs                 9514 non-null   category
 5   height                9514 non-null   float64 
 6   income                9514 non-null   float64 
 7   orientation           9514 non-null   category
 8   religion              9514 non-null   category
 9   sex                   9514 non-null   category
 10  smokes                9514 non-null   int32   
 11  status                9514 non-null   object  
 12  education_category    9514 non-null   category
 13  education_status      9514 non-null   category
 14  primary_ethnic_group  9514 non-null   category
 15  job_statu

In [51]:
# 60000 kayıtlı dosyanın height alanında eksik var
# df['height'].fillna(df.groupby('sex')['height'].transform('median'), inplace=True)

### Modelleme
Bu aşamada iki model deneyeceğiz:
1. Regression ile yaş tahmini yapacağız
2. Classification ile nesil sınıflandırması yapacağız.

#### 1. Yaş Tahmini

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

def algo_regression(x,y):
        #Bütün modelleri tanımlıyorum
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
        ab=AdaBoostRegressor()
        rf=RandomForestRegressor()
       
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb,ab,rf]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting',
                    'KNeighbors','Decision Tree','XGBRegressor', 'AdaBoost', 'Random Forest']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        #Hata ve doğruluk oranlarını bir tablo haline getirmek için bir dataframe oluşturuyorum
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
                
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
        #result adlı tabloya doğruluk ve hata oranlarımı yerleştiriyorum
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
       #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [53]:
x=df.drop("age", axis=1)
y=df[["age"]]
x=pd.get_dummies(x, drop_first=True)
x.shape, y.shape

((9514, 51), (9514, 1))

In [54]:
algo_regression(x,y)

,R_Squared,RMSE,MAE
Gradient Boosting,0.345624,7.711872,5.711485
Ridge,0.328827,7.810218,5.846177
Linear,0.328810,7.810318,5.847140
XGBRegressor,0.294024,8.010154,5.936407
Random Forest,0.251740,8.246549,6.107857
KNeighbors,0.121393,8.936003,6.590436
ElasticNet,0.050845,9.287836,7.010954
Lasso,0.026473,9.406326,7.096842
AdaBoost,-0.254174,10.676401,8.989670
Decision Tree,-0.306766,10.897952,7.864947


In [55]:
# Gradient Boosting	0.345544	7.712339	5.712359

#### 2. Sınıflandırma

In [56]:
df=df[(df['age'] <= 70) & (df['age'] >= 18)]
df.loc[(df['age'] >= 18) & (df['age'] <= 32), 'generation'] = 1 
df.loc[(df['age'] >= 22) & (df['age'] <= 47), 'generation'] = 2 
df.loc[(df['age'] >= 48) & (df['age'] <= 70), 'generation'] = 3

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9513 entries, 0 to 9513
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   age                   9513 non-null   float64 
 1   body_type             9513 non-null   category
 2   diet                  9513 non-null   category
 3   drinks                9513 non-null   category
 4   drugs                 9513 non-null   category
 5   height                9513 non-null   float64 
 6   income                9513 non-null   float64 
 7   orientation           9513 non-null   category
 8   religion              9513 non-null   category
 9   sex                   9513 non-null   category
 10  smokes                9513 non-null   int32   
 11  status                9513 non-null   object  
 12  education_category    9513 non-null   category
 13  education_status      9513 non-null   category
 14  primary_ethnic_group  9513 non-null   category
 15  job_statu

In [93]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

g = GaussianNB(priors=None, var_smoothing=1e-09)
b = BernoulliNB()
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
gb= GradientBoostingClassifier()
kn= KNeighborsClassifier()

def algo_classification(x, y):
    modeller=[g, b, l, d, r, gb, kn]
    isimler=["GaussianNB", "BernoulliNB", "LogisticRegression", "DecisionTreeClassifier", 
             "RandomForestClassifier", "GradientBoostingClassifier", "KNeighborsClassifier"]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state = 42)
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
    mdl=[]

    print("Veriler hazır modeller deneniyor")
    for model in modeller:
        print(model, " modeli eğitiliyor!..")
        model=model.fit(x_train,y_train)
        tahmin=model.predict((np.array(x_test) if model==kn else x_test))
        mdl.append(model)
        accuracy.append(accuracy_score(tahmin, y_test))
        precision.append(precision_score(tahmin, y_test, average='weighted'))
        recall.append(recall_score(tahmin, y_test, average='weighted'))
        f1.append(f1_score(tahmin, y_test, average='weighted'))
        print(confusion_matrix(tahmin, y_test))

    print("Eğitim tamamlandı.")
    
    metrics=pd.DataFrame(columns=["Accuracy", "Precision", "Recall", "F1", "Model"], index=isimler)
    metrics["Accuracy"] = accuracy
    metrics["Precision"] = precision  
    metrics["Recall"] = recall
    metrics["F1"] = f1
    metrics["Model"]=mdl

    metrics.sort_values("F1", ascending=False, inplace=True)

    print("")
    print("##########################################")
    print("En başarılı model: ", metrics.iloc[0].name)
    model=metrics.iloc[0,-1]
    tahmin=model.predict(np.array(x_test) if model==kn else x_test)
    print("Confusion Matrix:")
    print(confusion_matrix(tahmin, y_test))
    print("classification Report:")
    print(classification_report(tahmin, y_test))
    print("Diğer Modeller:")
    
    return metrics.drop("Model", axis=1)

In [85]:
x=df.drop(["age", "generation"], axis=1)
y=df[["generation"]]
x=pd.get_dummies(x, drop_first=True)
x.shape, y.shape

((9513, 51), (9513, 1))

In [87]:
y["generation"].value_counts()

generation
2.0    8234
3.0     745
1.0     534
Name: count, dtype: int64

In [89]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x2, y2 = smote.fit_resample(x, y)

In [94]:
algo_classification(x2,y2)

Veriler hazır modeller deneniyor
GaussianNB()  modeli eğitiliyor!..
[[   7    2    2]
 [  20  109  132]
 [1591 1566 1512]]
BernoulliNB()  modeli eğitiliyor!..
[[1412  282   43]
 [ 129  993  226]
 [  77  402 1377]]
LogisticRegression()  modeli eğitiliyor!..
[[1455 1420 1311]
 [ 163  257  335]
 [   0    0    0]]
DecisionTreeClassifier()  modeli eğitiliyor!..
[[1551  129   16]
 [  59 1378   97]
 [   8  170 1533]]
RandomForestClassifier()  modeli eğitiliyor!..
[[1577   97    8]
 [  36 1469   73]
 [   5  111 1565]]
GradientBoostingClassifier()  modeli eğitiliyor!..
[[1514  226   79]
 [  55 1230  189]
 [  49  221 1378]]
KNeighborsClassifier()  modeli eğitiliyor!..
[[1589  158   13]
 [  20 1288   52]
 [   9  231 1581]]
Eğitim tamamlandı.

##########################################
En başarılı model:  RandomForestClassifier
Confusion Matrix:
[[1577   97    8]
 [  36 1469   73]
 [   5  111 1565]]
classification Report:
              precision    recall  f1-score   support

         1.0       0.

,Accuracy,Precision,Recall,F1
RandomForestClassifier,0.933212,0.935020,0.933212,0.933691
KNeighborsClassifier,0.902247,0.915215,0.902247,0.904707
DecisionTreeClassifier,0.903056,0.906659,0.903056,0.904031
GradientBoostingClassifier,0.834244,0.842514,0.834244,0.836295
BernoulliNB,0.765432,0.782578,0.765432,0.770422
GaussianNB,0.329488,0.871465,0.329488,0.458460
LogisticRegression,0.346489,0.785266,0.346489,0.457061


## Sonuç

Bu projede hem yaş (regression) hem de çağ (classification) modelleri geliştirilmiştir.